### Computing PCA using RDDs

###  PCA

The vectors that we want to analyze have length, or dimension, of 365, corresponding to the number of 
days in a year.

We want to perform [Principle component analysis (PCA)](https://en.wikipedia.org/wiki/Principal_component_analysis)
on these vectors. There are two steps to this process:

1. Computing the covariance matrix: this is a  simple computation. However, it takes a long time to compute and it benefits from using an RDD because it involves all of the input vectors.
2. Computing the eigenvector decomposition. this is a more complex computation, but it takes a fraction of a second because the size to the covariance matrix is $365 \times 365$, which is quite small. We do it on the head node usin `linalg`

### Computing the covariance matrix
Suppose that the data vectors are the column vectors denoted $x$ then the covariance matrix is defined to be
$$
E(x x^T)-E(x)E(x)^T
$$

Where $x x^T$ is the **outer product** of $x$ with itself.

If the data that we have is $x_1,x_2,x_n$ then the estimates we use are:
$$
\hat{E}(x x^T) = \frac{1}{n} \sum_{i=1}^n x_i x_i^T,\;\;\;\;\;
\hat{E}(x) = \frac{1}{n} \sum_{i=1}^n x_i
$$

### `nan`s in arithmetic operations
* We store all of the measurements as single `bytearray` in a single column. Instead of using 365 columns.
* Why?
  1. Because serializing and desirializing is faster that way.
  1. Because numpy treats `nan` entries correctly:
      * In `numpy.nansum` `5+nan=5` while in dataframes `5+nan=nan`

In [1]:
import numpy as np
X=np.array([1,1,1,2])
print 'mean of',X,'=',np.mean(X)
print 'nanmean of',X,'=',np.nanmean(X)
X=np.array([1,1,np.NaN,2])
print 'mean of',X,'=',np.mean(X)
print 'nanmean of',X,'=',np.nanmean(X)

mean of [1 1 1 2] = 1.25
nanmean of [1 1 1 2] = 1.25
mean of [  1.   1.  nan   2.] = nan
nanmean of [  1.   1.  nan   2.] = 1.33333333333


#### When should you not use `np.nanmean` ?
Using `n.nanmean` is equivalent to assuming that choice of which elements to remove is independent of the values of the elements. 
* Example of bad case: suppose the larger elements have a higher probability of being `nan`. In that case `np.nanmean` will under-estimate the mean

#### Computing Cov matrix on vectors with NaNs
As it happens, we often get vectors $x$ in which some, but not all, of the entries are `nan`. 
Suppose that we want to compute the mean of the elements of $x$. If we use `np.mean` we will get the result `nan`. A useful alternative is to use `np.nanmean` which removes the `nan` elements and takes the mean of the rest.

#### Computing the covariance  when there are `nan`s
The covariance is a mean of outer products.

If the data that we have is $x_1,x_2,x_n$ then the estimates we use are:
$$
\hat{E}(x x^T) = \frac{1}{n} \sum_{i=1}^n x_i x_i^T,\;\;\;\;\;
\hat{E}(x) = \frac{1}{n} \sum_{i=1}^n x_i
$$

In [2]:
x1=np.array([1,np.NaN,3,4,5])
x2=np.array([2,3,4,np.NaN,6])
stacked=np.array([np.outer(x1,x1),np.outer(x2,x2)])
stacked

array([[[  1.,  nan,   3.,   4.,   5.],
        [ nan,  nan,  nan,  nan,  nan],
        [  3.,  nan,   9.,  12.,  15.],
        [  4.,  nan,  12.,  16.,  20.],
        [  5.,  nan,  15.,  20.,  25.]],

       [[  4.,   6.,   8.,  nan,  12.],
        [  6.,   9.,  12.,  nan,  18.],
        [  8.,  12.,  16.,  nan,  24.],
        [ nan,  nan,  nan,  nan,  nan],
        [ 12.,  18.,  24.,  nan,  36.]]])

In [3]:
np.nanmean(stacked,axis=0)

/Users/jc/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:1: RuntimeWarning: Mean of empty slice
  if __name__ == '__main__':


array([[  2.5,   6. ,   5.5,   4. ,   8.5],
       [  6. ,   9. ,  12. ,   nan,  18. ],
       [  5.5,  12. ,  12.5,  12. ,  19.5],
       [  4. ,   nan,  12. ,  16. ,  20. ],
       [  8.5,  18. ,  19.5,  20. ,  30.5]])

### Loading Data into Dataframe

In [5]:
import findspark
findspark.init()

from pyspark import SparkContext
#sc.stop()
## add after creating spark_PCA.py
sc = SparkContext(master="local[3]",pyFiles=['lib/numpy_pack.py','lib/computeStats.py','lib/spark_PCA.py'])

from pyspark import SparkContext
from pyspark.sql import *
sqlContext = SQLContext(sc)

In [6]:
import sys
sys.path.append('./lib')

import numpy as np
from numpy_pack import packArray,unpackArray
from spark_PCA import computeCov
from computeStats import computeOverAllDist, STAT_Descriptions

### Climate data

The data we will use here comes from [NOAA](https://www.ncdc.noaa.gov/). Specifically, it was downloaded from This [FTP site](ftp://ftp.ncdc.noaa.gov/pub/data/ghcn/daily/).

There is a large variety of measurements from all over the world, from 1870 will 2012.
in the directory `../../Data/Weather` you will find the following useful files:

* data-source.txt: the source of the data
* ghcnd-readme.txt: A description of the content and format of the data
* ghcnd-stations.txt: A table describing the Meteorological stations.



### Data cleaning

* Most measurements exists only for a tiny fraction of the stations and years. We therefor restrict our use to the following measurements:
```python
['TMAX', 'SNOW', 'SNWD', 'TMIN', 'PRCP', 'TOBS']
```

* 8 We consider only measurement-years that have at most 50 `NaN` entries

* We consider only measurements in the continential USA

* We partition the stations into 256 geographical rectangles, indexed from BBBBBBBB to SSSSSSSS. And each containing about 12,000 station,year pairs.

In [181]:
file_index='BBBBSSBB'
#file_index='BBBSBBBB'
filename='US_Weather_%s.csv.gz'%file_index
data_dir='../../Data/Weather'
del_files='%s/%s*'%(data_dir,filename[:-3])
print del_files
!rm $del_files
command="curl https://mas-dse-open.s3.amazonaws.com/Weather/small/%s > %s/%s"%(filename,data_dir,filename)
print command
!$command
!ls -lh $data_dir/$filename

../../Data/Weather/US_Weather_BBBSBBBB.csv*
curl https://mas-dse-open.s3.amazonaws.com/Weather/small/US_Weather_BBBSBBBB.csv.gz > ../../Data/Weather/US_Weather_BBBSBBBB.csv.gz
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 3732k  100 3732k    0     0  2195k      0  0:00:01  0:00:01 --:--:-- 3190k
-rw-r--r--  1 jc  staff   3.6M May  8 06:32 ../../Data/Weather/US_Weather_BBBSBBBB.csv.gz


In [2]:
data_dir='../../Data/Weather'
file_index='BBBBSSBB'

In [3]:
!gunzip $data_dir/$filename
filename="{0}/US_Weather_{1}.csv".format(data_dir, file_index)
!ls -lh $filename

gunzip: / is a directory
-rw-r--r--  1 jc  staff    13M May  8 20:34 ../../Data/Weather/US_Weather_BBBBSSBB.csv


In [7]:
import pickle
List=pickle.load(open(filename,'rb'))
len(List)

12884

In [8]:
df=sqlContext.createDataFrame(List)
print df.count()
df.show(5)

12884
+---------+--------+---------+-----------+-----------+------+--------------------+------+--------+
|elevation|latitude|longitude|measurement|    station|undefs|              vector|  year|   label|
+---------+--------+---------+-----------+-----------+------+--------------------+------+--------+
|    106.7| 43.4622| -76.4933|       TMAX|USC00306314|     0|[20 50 40 4C 80 4...|1926.0|BBBBSSBB|
|    106.7| 43.4622| -76.4933|       TMAX|USC00306314|     0|[00 00 A0 D3 00 0...|1927.0|BBBBSSBB|
|    106.7| 43.4622| -76.4933|       TMAX|USC00306314|     1|[80 4D 80 D4 E0 D...|1928.0|BBBBSSBB|
|    106.7| 43.4622| -76.4933|       TMAX|USC00306314|     1|[00 53 00 00 00 C...|1929.0|BBBBSSBB|
|    106.7| 43.4622| -76.4933|       TMAX|USC00306314|     2|[E0 54 E0 55 00 4...|1930.0|BBBBSSBB|
+---------+--------+---------+-----------+-----------+------+--------------------+------+--------+
only showing top 5 rows



In [9]:
sqlContext.registerDataFrameAsTable(df,'weather') #using older sqlContext instead of newer (V2.0) sparkSession

In [10]:
from time import time
t=time()

N=sc.defaultParallelism
print 'Number of executors=',N
print 'took',time()-t,'seconds'

Number of executors= 3
took 0.00124907493591 seconds


### Computing PCA for each measurement

In [11]:
measurements=['TMAX', 'SNOW', 'SNWD', 'TMIN', 'PRCP', 'TOBS']

#### Homework
The code below computes the covariance matrix using RDDs.
The code allows undefined entries and calculates the covariance without bias.

Your homework is to complete the missing parts in the code (Marked with `...`) so that it would calculate the covariance correctly.

In [12]:
#%%writefile lib/spark_PCA.py #once this works correctly, you should add it to the `lib` directory
# and use import

import numpy as np
from numpy import linalg as LA

def outerProduct(X):
    """Computer outer product and indicate which locations in matrix are undefined"""
    O=np.outer(X,X)
    N=1-np.isnan(O)
    return (O,N)

def sumWithNan(M1,M2):
    """Add two pairs of (matrix,count)"""
    (X1,N1)=M1
    (X2,N2)=M2
    N=N1+N2
    X=np.nansum(np.dstack((X1,X2)),axis=2)
    return (X,N)

def computeCov(RDDin):
    """computeCov recieves as input an RDD of np arrays, all of the same length, 
    and computes the covariance matrix for that set of vectors"""
    RDD=RDDin.map(lambda v:np.array(np.insert(v,0,1),dtype=np.float64)) 
                     # insert a 1 at the beginning of each vector so that the same 
                     #calculation also yields the mean vector
    
    OuterRDD=outerProduct(RDD.map(lambda x:outerProduct(x))) # separating the map and the reduce does not matter because of Spark uses lazy execution.
    
    (S,N)=sumWithNan(OuterRDD.reduce(lambda x,y:sumWithNan(x,y)))
    E=S[0,1:]
    NE=np.float64(N[0,1:])
    print 'shape of E=',E.shape,'shape of NE=',NE.shape
    Mean=E/NE
    O=S[1:,1:]
    NO=np.float64(N[1:,1:])
    Cov=O/NO-np.outer(Mean,Mean)  
    # Output also the diagnal which is the variance for each day
    Var=np.array([Cov[i,i] for i in range(Cov.shape[0])])
    return {'E':E,'NE':NE,'O':O,'NO':NO,'Cov':Cov,'Mean':Mean,'Var':Var}

#### Let's find out which station has the least missing data to work with

In [15]:
# Check the data of our region
Query="select distinct station FROM weather where station = 'USC00306184' group by station "
print Query
station = sqlContext.sql(Query)
station.show()

select distinct station FROM weather where station = 'USC00306184' group by station 
+-----------+
|    station|
+-----------+
|USC00306184|
+-----------+



In [190]:
stations = station.rdd.sortByKey(lambda x:x).collect()

In [191]:
station_codes = [s[0] for s in stations]#if s[1] < 100]
station_codes

[u'US1MAES0002',
 u'US1MAES0008',
 u'US1MAMD0007',
 u'US1MAMD0011',
 u'US1MAMD0012',
 u'US1MANF0001',
 u'US1MANF0005',
 u'US1MAWR0001',
 u'US1MAWR0013',
 u'USC00190120',
 u'USC00190166',
 u'USC00190218',
 u'USC00190257',
 u'USC00190408',
 u'USC00190535',
 u'USC00190538',
 u'USC00190551',
 u'USC00190562',
 u'USC00190582',
 u'USC00190593',
 u'USC00190595',
 u'USC00190655',
 u'USC00190768',
 u'USC00190801',
 u'USC00190998',
 u'USC00191097',
 u'USC00191099',
 u'USC00191103',
 u'USC00191110',
 u'USC00191323',
 u'USC00191344',
 u'USC00191447',
 u'USC00191561',
 u'USC00191563',
 u'USC00191582',
 u'USC00191589',
 u'USC00191622',
 u'USC00191950',
 u'USC00192107',
 u'USC00192150',
 u'USC00192530',
 u'USC00192806',
 u'USC00192812',
 u'USC00192975',
 u'USC00193052',
 u'USC00193110',
 u'USC00193224',
 u'USC00193229',
 u'USC00193270',
 u'USC00193295',
 u'USC00193401',
 u'USC00193402',
 u'USC00193405',
 u'USC00193429',
 u'USC00193624',
 u'USC00193625',
 u'USC00193640',
 u'USC00193690',
 u'USC00193702

#### Let's examine the stations further and see which one is a good candidate for our study

In [192]:
# we want stations with all types of measurements
Query="SELECT station,count(distinct year) as year,sum(undefs) as undefs "\
      "FROM weather "\
      "WHERE station in ('{0}') group by station "\
      "having count(distinct measurement) = 6 "\
      "order by undefs".format("','".join(station_codes))
#print Query
measurement = sqlContext.sql(Query)
measurement.show()

+-----------+----+------+
|    station|year|undefs|
+-----------+----+------+
|USC00190655|   1|    62|
|USC00199289|   1|    64|
|USC00194154|   3|   103|
|USC00198755|  23|   112|
|USC00193690|   3|   139|
|USC00190166|   7|   149|
|USC00198760|   7|   203|
|USC00193224|  15|   204|
|USC00191950|   2|   229|
|USC00191344|   4|   239|
|USC00194162|   4|   289|
|USC00196245|  27|   354|
|USC00198757|  40|   394|
|USC00193624|  52|   411|
|USC00193229|   7|   424|
|USC00197124|  13|   459|
|USW00014703|  30|   549|
|USC00194771|  28|   556|
|USC00198301|  26|   587|
|USC00195175|  33|   602|
+-----------+----+------+
only showing top 20 rows



#### It looks like some of the stations only have a few years of data collection.  Let's narrow down our list by selecting those with more than 50 years of data.

In [ ]:
USC00198301

In [194]:
years=measurement.rdd.filter(lambda row: row['year']>10)
years.take(20)

[Row(station=u'USC00198755', year=23, undefs=112),
 Row(station=u'USC00193224', year=15, undefs=204),
 Row(station=u'USC00196245', year=27, undefs=354),
 Row(station=u'USC00198757', year=40, undefs=394),
 Row(station=u'USC00193624', year=52, undefs=411),
 Row(station=u'USC00197124', year=13, undefs=459),
 Row(station=u'USW00014703', year=30, undefs=549),
 Row(station=u'USC00194771', year=28, undefs=556),
 Row(station=u'USC00198301', year=26, undefs=587),
 Row(station=u'USC00195175', year=33, undefs=602),
 Row(station=u'USC00194502', year=28, undefs=618),
 Row(station=u'USC00194580', year=47, undefs=713),
 Row(station=u'USC00190593', year=16, undefs=720),
 Row(station=u'USC00199316', year=48, undefs=846),
 Row(station=u'USC00190535', year=51, undefs=852),
 Row(station=u'USC00196783', year=52, undefs=951),
 Row(station=u'USW00014753', year=120, undefs=973),
 Row(station=u'USC00191582', year=17, undefs=1015),
 Row(station=u'USC00193890', year=44, undefs=1056),
 Row(station=u'USC00198580',

#### USC00303851 seems to have one of the higher counts of 'year' with fewer undefs.  This may be a good candidate for our study

In [195]:
#station = 'USC00301732'
station = 'USC00198301'
#station = 'USC00303851'
Query = "select distinct year from weather where station = '{0}' group by year order by year".format(station)
print Query
top_station = sqlContext.sql(Query)
top_station.show()

select distinct year from weather where station = 'USC00198301' group by year order by year
+------+
|  year|
+------+
|1931.0|
|1932.0|
|1933.0|
|1934.0|
|1935.0|
|1936.0|
|1937.0|
|1938.0|
|1939.0|
|1940.0|
|1941.0|
|1942.0|
|1943.0|
|1944.0|
|1945.0|
|1946.0|
|1947.0|
|1948.0|
|1949.0|
|1950.0|
+------+
only showing top 20 rows



In [196]:
top = top_station.rdd.map(lambda row: row['year']).collect()

In [197]:
top

[1931.0,
 1932.0,
 1933.0,
 1934.0,
 1935.0,
 1936.0,
 1937.0,
 1938.0,
 1939.0,
 1940.0,
 1941.0,
 1942.0,
 1943.0,
 1944.0,
 1945.0,
 1946.0,
 1947.0,
 1948.0,
 1949.0,
 1950.0,
 1951.0,
 1952.0,
 1953.0,
 1954.0,
 1955.0,
 1956.0]

In [ ]:
from numpy import linalg as LA
STAT={}  # dictionary storing the statistics for each measurement
Clean_Tables={}

for meas in measurements:
    t=time()
    Query="SELECT * FROM weather WHERE measurement = '{1}'".format(station,meas)
    print Query
    df1 = sqlContext.sql(Query)
    data=df1.rdd.map(lambda row: unpackArray(row['vector'],np.float64))
    #data=df1.rdd.map(lambda row: unpackArray(row['vector'],np.float16))
    #get basic statistics
    print 'data',type(data)
    temp = data.map(lambda row:sum(np.isnan(row)))
    print 'temp', temp
    temp2 = temp.sample(False,0.01)
    print 'temp2', temp2
    temp3 = temp2.collect()
    print 'temp3', type(temp3)
    temp4=np.array(temp3)
    print 'temp4', temp4
    

    STAT[meas]=computeOverAllDist(data)   # Compute the statistics 

    # compute covariance matrix
    OUT=computeCov(data)

    #find PCA decomposition
    eigval,eigvec=LA.eig(OUT['Cov'])

    # collect all of the statistics in STAT[meas]
    STAT[meas]['eigval']=eigval
    STAT[meas]['eigvec']=eigvec
    STAT[meas].update(OUT)

    print 'time for',meas,'is',time()-t

In [163]:
from pickle import dump
filename=data_dir+'/STAT_%s.pickle'%file_index
dump((STAT,STAT_Descriptions),open(filename,'wb'))
!ls -lrth $data_dir

total 288952
-rw-r--r--    1 jc  staff   218B May  1 08:46 data-source.txt
-rw-r--r--    1 jc  staff   2.1M May  1 08:46 stations.pkl.gz
-rw-r--r--    1 jc  staff   270B May  1 08:46 ghcnd-version.txt
-rw-r--r--    1 jc  staff   7.4M May  1 08:46 ghcnd-stations_buffered.txt
-rw-r--r--    1 jc  staff   8.0M May  1 08:46 ghcnd-stations.txt
-rw-r--r--    1 jc  staff    22K May  1 08:46 ghcnd-readme.txt
drwxr-xr-x  404 jc  staff    13K May  1 08:46 decon_BBBSBBBB_TOBS.parquet
drwxr-xr-x  404 jc  staff    13K May  1 08:46 decon_BBBSBBBB.parquet
drwxr-xr-x   10 jc  staff   340B May  1 08:46 US_Weather_BBBSBBBB.parquet
-rw-r--r--    1 jc  staff   3.6M May  1 08:46 US_Weather_BBBSBBBB.csv.gz
-rw-r--r--    1 jc  staff    13M May  1 08:46 US_Weather_BBBSBBBB.csv
-rw-r--r--    1 jc  staff    16M May  1 08:46 STAT_BBBSBBBB.pickle.gz
-rw-r--r--    1 jc  staff    78M May  5 22:33 STAT_BBBSBBBB.pickle
drwxr-xr-x  404 jc  staff    13K May  5 22:39 decon_BBBSBBBB_SNWD.parquet
drwxr-xr-x  404 jc  staff 